In [1]:
import os, sys, time, math
import tensorflow as tf
import tensorlayer as tl
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# import network, img_io

eps = 1.0/255.0


## 1. Import LDR images 
these HDR images are derived from HDR image dataset from google. 
the code used for this procedure is function from MATLAB


In [ ]:
def import_images():
    # read data from <data> folder
    
    return X, y

In [ ]:
X = # simulated LDR images 152, 1208, 768, 3
y = # ground truth HDR images, 152, 4208 3120 3

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

## 2. Preprocess the images 
Convert the images into smaples??? or just image before the filters 

## 3. create input pipeline for generating training/testing batches

In [ ]:
# this can be either X_train/y_train or X_test/y_test, so we make a placeholder that we can feed into:
X_train_or_test = tf.placeholder(tf.float32, [None, 1028, 768, 3], name='input_LDR')
y_train_or_test = tf.placeholder(tf.float32, [None, 4208, 3120, 3], name='ground_truth_HDR')

batch_size = 4

# create a tf dataset, from which we can generate batches
dataset = tf.data.Dataset.from_tensor_slices((X_train_or_test, y_train_or_test))
dataset = dataset.batch(batch_size).repeat(None)

batch_generator = dataset.make_initializable_iterator()
X_batch, y_batch = batch_generator.get_next()  # batches symbolically generated

In [ ]:
X_batch

## 4. create three pixel-level filters 

In [ ]:
train_filters = True

input_filters = tf.Variable(np.ones([1028, 768, 3]), dtype=tf.float32, trainable=train_filters)

In [ ]:
input_filters

## 5. generate the filtered images


In [3]:
X_filtered = tf.multiply(X_batch, input_filters)

In [ ]:
X_filtered

## 6. propagate back to the image plane: filtered LDR 


In [ ]:

# add noise:
image = X_filtered + tf.random.normal([1028, 768, 3], mean=0, stddev=0.05)

In [ ]:
image

 ## 7. process the filtered image through a Physical CNN to optimize filters

- ### Pass through Digital layer to get HDR output image.
- ### use the same loss as the Digital layer. 
loss is illumination + reflectance loss
I just need revise the hdrcnn_predict.py into a function, which return the HDR image and the loss?

revise get_final to return not only the HDR imgae, but also the cost. 

or should I define my own loss? 

FLAGS.sep_loss is the flag to separate loss into illumination and reflectance


## Define the network 

### Cost function 

In [5]:
# Learning parameters
tf.flags.DEFINE_float("num_epochs",         "100.0",   "Number of training epochs")
tf.flags.DEFINE_float("learning_rate",      "0.00005", "Starting learning rate for Adam optimizer")
tf.flags.DEFINE_integer("batch_size",       "4",       "Batch size for training")

tf.flags.DEFINE_float("train_size",         "0.99",    "Fraction of data to use for training, the rest is validation data")

tf.flags.DEFINE_bool("sep_loss",            "true",    "Use illumination + reflectance loss")
tf.flags.DEFINE_float("lambda_ir",          "0.5",     "Reflectance weight for the ill+refl loss")

In [6]:
def calculate_loss(x, y_):
    """
    params required to calculate loss:
    y_
    x
    eps
    FLAGS.sep_loss
    FLAGS.lambda_ir
    
    
    """
    
    # Loss separated into illumination and reflectance terms
    if FLAGS.sep_loss:
        y_log_ = tf.log(y_+eps)  
        x_log = tf.log(tf.pow(x, 2.0)+eps)

        # Luminance
        lum_kernel = np.zeros((1, 1, 3, 1))
        lum_kernel[:, :, 0, 0] = 0.213
        lum_kernel[:, :, 1, 0] = 0.715
        lum_kernel[:, :, 2, 0] = 0.072
        y_lum_lin_ = tf.nn.conv2d(y_, lum_kernel, [1, 1, 1, 1], padding='SAME')
        y_lum_lin = tf.nn.conv2d(tf.exp(y)-eps, lum_kernel, [1, 1, 1, 1], padding='SAME')
        x_lum_lin = tf.nn.conv2d(x, lum_kernel, [1, 1, 1, 1], padding='SAME')

        # Log luminance
        y_lum_ = tf.log(y_lum_lin_ + eps)
        y_lum = tf.log(y_lum_lin + eps)
        x_lum = tf.log(x_lum_lin + eps)

        # Gaussian kernel
        nsig = 2
        filter_size = 13
        interval = (2*nsig+1.)/(filter_size)
        ll = np.linspace(-nsig-interval/2., nsig+interval/2., filter_size+1)
        kern1d = np.diff(st.norm.cdf(ll))
        kernel = kernel_raw/kernel_raw.sum()

        # Illumination, approximated by means of Gaussian filtering
        weights_g = np.zeros((filter_size, filter_size, 1, 1))
        weights_g[:, :, 0, 0] = kernel
        y_ill_ = tf.nn.conv2d(y_lum_, weights_g, [1, 1, 1, 1], padding='SAME')
        y_ill = tf.nn.conv2d(y_lum, weights_g, [1, 1, 1, 1], padding='SAME')
        x_ill = tf.nn.conv2d(x_lum, weights_g, [1, 1, 1, 1], padding='SAME')

        # Reflectance
        y_refl_ = y_log_ - tf.tile(y_ill_, [1,1,1,3])
        y_refl = y - tf.tile(y_ill, [1,1,1,3])
        x_refl = x - tf.tile(x_ill, [1,1,1,3])

        cost =              tf.reduce_mean( ( FLAGS.lambda_ir*tf.square( tf.subtract(y_ill, y_ill_) ) + (1.0-FLAGS.lambda_ir)*tf.square( tf.subtract(y_refl, y_refl_) ) )*msk )
        cost_input_output = tf.reduce_mean( ( FLAGS.lambda_ir*tf.square( tf.subtract(x_ill, y_ill_) ) + (1.0-FLAGS.lambda_ir)*tf.square( tf.subtract(x_refl, y_refl_) ) )*msk )
    else:
        cost =              tf.reduce_mean( tf.square( tf.subtract(y, tf.log(y_+eps) )*msk ) )
        cost_input_output = tf.reduce_mean( tf.square( tf.subtract(tf.log(y_+eps), tf.log(tf.pow(x, 2.0)+eps) )*msk ) );
    return [cost, cost_input_output]

In [ ]:
loss, _ = calculate_loss(x=image, y_=y_batch)

In [ ]:
train_op = tf.train.GradientDescentOptimizer(learning_rate=Flags.learning_rate).minimize(loss)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

## 8. Train

In [ ]:
def plot_examples(batch):
    plt.figure(figsize=(30, 30))
    for i in range(4):
        plt.subplot(2, 2, i+1)
        plt.imshow(batch[i])
        plt.colorbar()
        plt.axis('off')
    plt.show()

In [ ]:
# this code tells our batch_generator to generate training batches:
sess.run(batch_generator.initializer, feed_dict={X_train_or_test: X_train, y_train_or_test: y_train})

In [ ]:
plot_examples(image.eval())

In [ ]:
for i in range(1000):
    _, loss_i = sess.run([train_op, loss])
    if i%1000 == 0:
        print(loss_i)
        print(input_filters.eval())

In [ ]:
# plot examples after the training
plot_examples(image.eval())

## 9. Results

In [ ]:
def compare_results():
    """plot:
    simulated_LDR
    simulated_HDR AFTER training
    simulated_HDR BEFORE training    
    ground_truth_HDR
    
    """ 
    
    pass